# Setting up a Ray cluster with SmartSim

## 1. Start the cluster
We set up a SmartSim experiment, which will handle the launch of the Ray cluster.

First we import the relevant modules.

In [1]:
%env SMARTSIM_LOG_LEVEL developer

env: SMARTSIM_LOG_LEVEL=developer


In [2]:
import numpy as np
import time
import argparse
import os

from ray.tune.progress_reporter import JupyterNotebookReporter
import ray
from ray import tune
import ray.util

from smartsim import Experiment
from smartsim.ray import RayCluster

from smartsim.launcher import slurm

NUM_WORKERS = 3
alloc=None
launcher='slurm'
#alloc=slurm.get_allocation(nodes=1+NUM_WORKERS, time="12:00:00", options={"ntasks": str(1+NUM_WORKERS), "partition": "spider", "C": "V100"})
alloc=slurm.get_allocation(nodes=1+NUM_WORKERS, time="12:00:00", options={"ntasks": str(1+NUM_WORKERS)})

13:59:56 nid00000 smartsim.launcher.slurm.slurm[48868] DEBUG Allocation settings: -N 4 -J SmartSim -t 12:00:00 --ntasks=4
13:59:56 nid00000 smartsim.launcher.util.shell[48868] DEBUG Executing Popen cmd: /opt/slurm/20.11.5/bin/salloc --no-shell -N 4 -J SmartSim -t 12:00:00 --ntasks=4
14:00:26 nid00000 smartsim.launcher.slurm.slurm[48868] INFO Allocation successful with Job ID: 1461743


In [3]:
exp = Experiment("ray-cluster", launcher=launcher)
cluster = RayCluster(name="ray-cluster", run_args={}, path='',
                     launcher=launcher, workers=NUM_WORKERS, alloc=alloc, batch=False)

if cluster.batch:
    cluster.head_model.batch_settings.add_preamble( ["source ~/.bashrc", "conda activate smartsim"])
    if NUM_WORKERS:
        cluster.worker_model.batch_settings.add_preamble ( ["source ~/.bashrc", "conda activate smartsim"])

exp.generate(cluster, overwrite=True)

14:00:32 nid00000 smartsim.generation.generator[48868] INFO Working in previously created experiment


In [4]:
exp.start(cluster, block=False, summary=False)

14:00:33 nid00000 smartsim.launcher.taskManager[48868] DEBUG Starting Task Manager
14:00:33 nid00000 smartsim.launcher.util.shell[48868] DEBUG Executing command: srun --output /lus/cls01029/arigazzi/smartsim-dev/SmartSim/tutorials/05_starting_ray/ray-cluster/head/head.out --error /lus/cls01029/arigazzi/smartsim-dev/SmartSim/tutorials/05_starting_ray/ray-cluster/head/head.err --job-name head-CCY76DP17ELD --jobid 1461743 --unbuffered --nodes=1 --time=01:00:00 --ntasks=1 --ntasks-per-node=1 python /lus/cls01029/arigazzi/smartsim-dev/SmartSim/smartsim/ray/raystarter.py --port=6780 --redis-password=80f909ec-a442-4493-9712-71d10bec7439
14:00:33 nid00000 smartsim.launcher.taskManager[48868] DEBUG Starting Task 49029
14:00:35 nid00000 smartsim.launcher.util.shell[48868] DEBUG Executing Popen cmd: /opt/slurm/20.11.5/bin/sacct --noheader -p --format=jobname,jobid
14:00:36 nid00000 smartsim.control.controller[48868] DEBUG Launching head
14:00:36 nid00000 smartsim.launcher.util.shell[48868] DEBUG 

## 2. Start the ray driver script

In [5]:
ray.util.connect(cluster.head_model.address +":10001")

In [6]:
print('''This cluster consists of
    {} nodes in total
    {} CPU resources in total
'''.format(len(ray.nodes()), ray.cluster_resources()['CPU']))

This cluster consists of
    4 nodes in total
    152.0 CPU resources in total



In [7]:
tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
        "num_gpus": 0,
        "lr": tune.grid_search(np.linspace (0.001, 0.01, 2).tolist()),
        "log_level": "ERROR",
    },
    local_dir="/lus/scratch/arigazzi/ray_local/",
    verbose=1,
    fail_fast=True,
    log_to_file=True,
)

(pid=25010) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=25010) Instructions for updating:
(pid=25010) non-resource variables are not supported in the long term


(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 5.8/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 3.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 2/2 (1 PENDING, 1 RUNNING)
(pid=25010) 
(pid=25010) 


(pid=25038) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=25038) Instructions for updating:
(pid=25038) non-resource variables are not supported in the long term
(pid=25038) 2021-07-20 11:37:15,728	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=25036) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=25036) Instructions for updating:
(pid=25036) non-resource variables are not supported in the long term
(pid=25037) WARNING:tensorflow:From /lus/scratch/ariga

(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 6.3/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 6.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 2/2 (2 RUNNING)
(pid=25010) 
(pid=25010) 


(pid=95527) 2021-07-20 11:37:27,106	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=95526) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=95526) Instructions for updating:
(pid=95526) non-resource variables are not supported in the long term
(pid=95528) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=95528) Instructions for updating:
(pid=95528) non-resource variables are not supported in the long term


(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 6.3/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 6.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 2/2 (2 RUNNING)
(pid=25010) 
(pid=25010) 
(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 6.3/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 3.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 2/2 (1 RUNNING, 1 TERMINATED)
(pid=25010) 
(pid=25010) 
(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 5.6/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigaz

(pid=25010) 2021-07-20 11:37:49,901	INFO tune.py:549 -- Total run time: 114.79 seconds (37.26 seconds for the tuning loop).


Instructions for updating:
non-resource variables are not supported in the long term


## 3. Stop cluster and release allocation

In [5]:
if alloc:
    slurm.release_allocation(alloc)

In [5]:
exp.stop(cluster)

In [8]:

tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
    #    "num_gpus": 0,
        "lr": tune.grid_search(np.linspace (0.001, 0.01, 200).tolist()),
        "log_level": "ERROR",
    #    "num_cpus_per_worker": 1,
    },
    local_dir="/lus/scratch/arigazzi/ray_local/",
    verbose=1,
    #fail_fast=True,
    #log_to_file=True,
    #progress_reporter = JupyterNotebookReporter(True),
)

(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 6.0/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 3.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 200/200 (199 PENDING, 1 RUNNING)
(pid=25010) 
(pid=25010) 


(pid=25031) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=25031) Instructions for updating:
(pid=25031) non-resource variables are not supported in the long term
(pid=25030) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=25030) Instructions for updating:
(pid=25030) non-resource variables are not supported in the long term
(pid=25032) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is 

(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 11.1/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 150.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 200/200 (150 PENDING, 50 RUNNING)
(pid=25010) 
(pid=25010) 


(pid=96075) 2021-07-20 11:40:30,620	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=96090) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=96090) Instructions for updating:
(pid=96090) non-resource variables are not supported in the long term
(pid=96097) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=96097) Instructions for updating:
(pid=96097) non-resource variables are not supported in the long term
(pid=96102) WARNING:tensorflow:From /lus/scratch/ariga

(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 14.0/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 150.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 200/200 (150 PENDING, 50 RUNNING)
(pid=25010) 
(pid=25010) 
(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 14.0/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 150.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 200/200 (150 PENDING, 50 RUNNING)
(pid=25010) 
(pid=25010) 


...

(pid=1704) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=1704) Instructions for updating:
(pid=1704) non-resource variables are not supported in the long term
(pid=1707) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=1707) Instructions for updating:
(pid=1707) non-resource variables are not supported in the long term
(pid=108282) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is depre

(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 14.3/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 135.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 200/200 (45 RUNNING, 155 TERMINATED)
(pid=25010) 
(pid=25010) 


(pid=2230) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=2230) Instructions for updating:
(pid=2230) non-resource variables are not supported in the long term
(pid=2231) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=2231) Instructions for updating:
(pid=2231) non-resource variables are not supported in the long term
(pid=108924) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is depre

(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 14.3/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 123.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 200/200 (41 RUNNING, 159 TERMINATED)
(pid=25010) 
(pid=25010) 
(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 12.5/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 93.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=25010) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=25010) Number of trials: 200/200 (31 RUNNING, 169 TERMINATED)
(pid=25010) 
(pid=25010) 
(pid=25010) == Status ==
(pid=25010) Memory usage on this node: 10.0/187.6 GiB
(pid=25010) Using FIFO scheduling algorithm.
(pid=25010) Resources requested: 60.0/152 CPUs, 0/0 GPUs, 0.0/512.14 GiB heap, 0.0/223.48 GiB objects
(pid=2501

(pid=25010) 2021-07-20 11:43:19,814	INFO tune.py:549 -- Total run time: 174.98 seconds (174.73 seconds for the tuning loop).


In [10]:
analysis = tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
        "lr": tune.grid_search([0.001, 0.002, 0.003, 0.004, 0.005]),
    },
    progress_reporter = JupyterNotebookReporter(True)
)

<IPython.core.display.HTML object>


(pid=110180) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=110180) Instructions for updating:
(pid=110180) non-resource variables are not supported in the long term
(pid=40586) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=40586) Instructions for updating:
(pid=40586) non-resource variables are not supported in the long term
(pid=110180) 2021-05-25 10:04:22,637	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=42854) WARNING:tensorflow:From /lus/scratch/ar

(pid=27027) 
(pid=27027) <IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


(pid=27027) 2021-05-25 10:04:54,258	INFO tune.py:549 -- Total run time: 34.48 seconds (34.15 seconds for the tuning loop).
(pid=42854) 2021-05-25 10:04:54,221	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=42854) Traceback (most recent call last):
(pid=42854)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=42854)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=42854)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=42854)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=42854)     return method(__ray_actor, *args, **kwargs)
(pid=42854)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/actor.py", line 1001, in __ray_terminate__
(pid=42854)     ray.actor.exit_actor()
(pid=42854)   File "/lus/scratch/arigazzi/anaconda3/envs/smart